In [2]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
# Method to get all the revelent player attributes for every player currently signed on every NFL team
# The CBS Sports website is used as the source

def GetPlayterAttributes(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from cbssports.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/{team[1]}/{team[0]}/roster/')
    
    # If we get a normal response proceed with data scraping
    if CBSResponse.status_code == 200:
                 
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        
        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the relevant data from the DataFrame(offense, defense, and special teams)
        OffenseDF = DataFrames[0][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        DefenseDF = DataFrames[1][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        SpecialDF = DataFrames[2][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        
        # Combine the three DataFrames into one
        FullTeamDF = pd.concat([OffenseDF, DefenseDF, SpecialDF], axis = 0, ignore_index = True)
        # print(FullTeamDF.head(20))
        
        # Method to clean the player names since CBS puts player news directly in the name cell
        def CleanNameCell(NameText):
            # Split the name cell into a list
            NameText = NameText.split(' ')
            
            # If the name cell doesn't contain player news and no name postfix
            if len(NameText) == 3:
                # Remove the first initial by selecting the rest of the elements
                NameText = NameText[1:]
                # Extract the first name from the first-last name string
                NameText[0] = NameText[0][len(NameText[1]):]
                # Create new string with first and last name
                NameText = ' '.join(NameText)
                
            # The name cell contains a name postfix
            elif len(NameText) == 5:
                # Create new list to store the frist name, last name, postfix
                NameTextPostfix = []
                # Extract the first name from the first-suffix name string, add it to the list
                NameTextPostfix.append(NameText[2][len(NameText[4]):])
                # Add the last name to the list
                NameTextPostfix.append(NameText[1])
                # Add the postfix to the list
                NameTextPostfix.append(NameText[4])
                # Turn the list into a string
                NameText = ' '.join(NameTextPostfix)

            # Otherwise the player name cell contains player news, injury status
            else:
                # Select only the capitilized words from the player cell
                NameTextCapitalized = [s for s in NameText if s and not s[0].islower()]
                # Select the first inital which is always the first item in NameText when there is news, injury status
                First = NameTextCapitalized[0][0]
                # Select the last which is always the second item in NameText when there is news, injury status
                LastName =  NameTextCapitalized[1]
                # Create postfix varaible in case player name has it
                Postfix = ''
                PostfixList = {'Jr.', 'II','III', 'IV'} # set of possible postfixes

                # Iterate through the items in NameTextCapitalzied from the third item to the end
                for s in range(3, (len(NameTextCapitalized) - 1)):
                    # Make sure we're always comparing to the first letter of current item
                    FirstTemp = NameTextCapitalized[s][0]
                    # If the current element first char is the first initial and the next element is last name
                    if NameTextCapitalized[s][0] == FirstTemp and NameTextCapitalized[s + 1].replace(":", "") == LastName:
                        # If we haven't reached the second to last elementand the next next element is a postfix
                        if NameTextCapitalized[s + 2] in PostfixList:
                            # Set the postfix variable to the next next element
                            Postfix = NameTextCapitalized[s + 2].replace(".", "") # remove the period from the postfix

                        # Set the first and last name 
                        First = NameTextCapitalized[s]
                        LastName = NameTextCapitalized[s + 1].replace(":", "") # remove the possible colon from the last name
                        # No break point because the first instance of the full name might not contain a postfix
                        
                        
                # Recreats NameText with the first name, last name
                NameText = [First, LastName]
                # If a postfix was found, add it to the list
                if Postfix:
                    NameText.append(Postfix)
                
                NameText = ' '.join(NameText)

            
            return NameText
        

        # Apply the method to the player names column
        FullTeamDF['Player'] = FullTeamDF['Player'].apply(CleanNameCell)  
        # Sort the DataFrame by player name
        FullTeamDF = FullTeamDF.sort_values(by = ['Player'], ignore_index = True) 
        
        # Capitilize the team name for consistency
        TeamNameCapitalized = (team[0].replace("-", " ")).split(' ')
        # We have to account the 49ers and their team name starting with a number
        TeamNameCapitalized = [s.capitalize() if not s[0].isdigit() else s for s in TeamNameCapitalized]
        TeamNameCapitalized = ' '.join(TeamNameCapitalized)
        
        # Create a .csv file with all the player names and their cap hits
        FullTeamDF.to_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        FullTeamCSV = pd.read_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', header = None)
        FullTeamCSV.columns = ['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']
        print(FullTeamCSV)
        
        return 0

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1


In [15]:
# Method to get the revelent attributes for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllPlayerAttributes():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    # Take the first two columns of each row, convert to a list, append to TeamList
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetPlayterAttributes(team)
    

GetAllPlayerAttributes()

Request succeeded with status code 200
                     Player  POS    HT   WT        DOB (AGE) EXP
0              Aaron Brewer   LS   6-5  232    7/5/1990 (34)  13
1           Andre Baccellia   WR  5-10  175    1/7/1997 (27)   2
2            Andre Chachere   DB   6-0  197   2/10/1996 (28)   4
3          Austen Pleasants   OT   6-7  330   8/22/1997 (26)   1
4                BJ Ojulari   LB   6-2  248    4/5/2002 (22)   2
5                Ben Stille   DE   6-4  296  11/12/1997 (26)   2
6        Bernhard Seikovits   TE   6-5  262   7/24/1997 (26)   1
7             Bilal Nichols   DT   6-3  313   9/14/1996 (27)   7
8            Blake Gillikin    P   6-2  196   1/21/1998 (26)   5
9          Blake Whiteheart   TE   6-4  247   3/22/2000 (24)   2
10              Bobby Price   DB   6-3  208   4/25/1998 (26)   5
11              Budda Baker   SS  5-10  195   1/10/1996 (28)   8
12           Cameron Thomas   LB   6-4  267    7/1/2000 (24)   3
13         Carter O'Donnell   OG   6-5  305  12/23/

Request succeeded with status code 200
                     Player  POS    HT   WT        DOB (AGE) EXP
0               Adisa Isaac   LB   6-4  254   10/4/2001 (22)   R
1            Andrew Vorhees   OG   6-6  310   1/21/1999 (25)   1
2      Ar'Darius Washington  SAF   5-8  176   11/2/1999 (24)   3
3             Arthur Maulet   CB   5-9  190   7/13/1993 (30)   8
4                Beau Brade   DB   6-0  203   1/29/2002 (22)   R
5             Ben Cleveland   OG   6-6  370   8/25/1998 (25)   4
6          Brandon Stephens   CB   6-1  215  12/29/1997 (26)   4
7               Brent Urban   DE   6-7  309    5/5/1991 (33)  11
8      Broderick Washington   DT   6-2  314   12/4/1996 (27)   5
9             C.J. Ravenell   DE   6-5  280                —   R
10            Charlie Kolar   TE   6-6  257   2/10/1999 (25)   3
11              Chris Board   LB   6-2  229   7/23/1995 (28)   7
12            Chris Collier   RB  5-11  203   3/24/2000 (24)   R
13        Christian Matthew   CB   6-2  195  10/26/

Request succeeded with status code 200
                  Player  POS    HT   WT        DOB (AGE) EXP
0       A'Shawn Robinson   DT   6-3  320   3/21/1995 (29)   9
1           Adam Thielen   WR   6-2  195   8/22/1990 (33)  11
2              Alex Cook  SAF   6-1  196   5/21/1999 (25)   2
3            Amare Barno   LB   6-6  245   4/26/1999 (25)   3
4            Andrew Raym    C   6-4  327    5/2/2001 (23)   R
5            Andy Dalton   QB   6-2  220  10/29/1987 (36)  14
6         Austin Corbett    C   6-4  306    9/5/1995 (28)   7
7          Badara Traore   OT   6-7  320   3/12/1997 (27)   1
8      Brady Christensen   OG   6-6  300   9/27/1996 (27)   4
9            Bryce Young   QB  5-10  204   7/25/2001 (22)   2
10             Cade Mays   OG   6-6  325   4/26/1999 (25)   3
11              Cam Gill   LB   6-3  240  12/14/1997 (26)   5
12              Cam Sims   WR   6-5  214    1/6/1996 (28)   6
13       Chandler Wooten   LB   6-3  230   4/25/1999 (25)   3
14       Chandler Zavala   OG  

Request succeeded with status code 200
                 Player  POS    HT   WT        DOB (AGE) EXP
0           Aaron Casey   LB   6-1  231   6/12/2000 (24)   R
1   Akeem Davis-Gaither   LB   6-2  228   9/21/1997 (26)   5
2            Alex Cappa   OG   6-6  310   1/27/1995 (29)   7
3          Allan George   CB  5-11  191   7/16/1999 (24)   1
4          Amarius Mims   OT   6-7  340  10/14/2002 (21)   R
5        Andrei Iosivas   WR   6-3  205  10/15/1999 (24)   2
6       Austin McNamara    P   6-5  204   2/24/2001 (23)   R
7             B.J. Hill   DT   6-3  311   4/20/1995 (29)   7
8          Brad Robbins    P   6-1  205   10/6/1998 (25)   2
9          Cal Adomitis   LS   6-2  238    7/9/1998 (26)   3
10           Cam Grandy   TE   6-5  253  11/19/1999 (24)   R
11     Cam Taylor-Britt   CB  5-11  202  10/15/1999 (24)   3
12       Cameron Sample   DE   6-3  272   9/20/1999 (24)   4
13       Cedric Johnson   DE   6-3  260    9/6/2002 (21)   R
14        Charlie Jones   WR   6-0  183  10/29

Request succeeded with status code 200
                  Player  POS    HT   WT        DOB (AGE) EXP
0          Adam Trautman   TE   6-5  253    2/5/1997 (27)   5
1           Alex Forsyth    C   6-4  312   2/13/1999 (25)   2
2        Alex Palczewski   OT   6-6  315    8/3/1999 (24)   2
3         Alex Singleton   LB   6-2  240   12/7/1993 (30)   6
4            Andre Smith   LB   6-0  240   4/20/1997 (27)   5
5        Angelo Blackson   DE   6-4  318  11/14/1992 (31)  10
6              Art Green   CB   6-1  201   4/29/2000 (24)   1
7          Audric Estime   RB  5-11  227    9/6/2003 (20)   R
8         Baron Browning   LB   6-3  240   2/19/1999 (25)   4
9             Ben Powers   OG   6-4  310  10/29/1996 (27)   6
10          Blake Watson   RB   5-9  195  10/14/1999 (24)   R
11                Bo Nix   QB   6-2  217   2/25/2000 (24)   R
12       Brandon Johnson   WR   6-2  195   7/26/1998 (25)   3
13         Brandon Jones  SAF   6-1  191    4/2/1998 (26)   5
14     Brandon Matterson   DL  

...

Request succeeded with status code 200
                   Player  POS    HT   WT        DOB (AGE) EXP
0              A.T. Perry   WR   6-5  205  10/26/1999 (24)   2
1           Adam Prentice   RB   6-0  245   1/16/1997 (27)   4
2          Alontae Taylor   CB   6-1  199   12/3/1998 (25)   3
3            Alvin Kamara   RB  5-10  215   7/25/1995 (28)   8
4           Anfernee Orji   LB   6-2  230   10/6/2000 (23)   1
5             Blake Grupe    K   5-7  156   11/5/1998 (25)   2
6            Bryan Bresee   DT   6-5  305   10/6/2001 (22)   2
7               Bub Means   WR   6-2  215   1/10/2001 (23)   R
8          Cameron Jordan   DE   6-4  287   7/10/1989 (35)  14
9         Carl Granderson   DE   6-5  261  12/18/1996 (27)   6
10     Cedrick Wilson Jr.   WR   6-2  197  11/20/1995 (28)   7
11             Cesar Ruiz   OG   6-4  316   6/14/1999 (25)   5
12          Charlie Smyth    K   6-4  210   6/26/2001 (23)   R
13            Chase Young   DE   6-5  264   4/14/1999 (25)   5
14            Ch

Request succeeded with status code 200
                      Player  POS    HT   WT        DOB (AGE) EXP
0              Aaron Rodgers   QB   6-2  223   12/2/1983 (40)  20
1              Al Blades Jr.  SAF   6-1  195  10/24/1999 (24)   R
2               Allen Lazard   WR   6-5  227  12/11/1995 (28)   6
3             Andrew Peasley   QB   6-2  218    1/6/2000 (24)   R
4               Ashtyn Davis  SAF   6-1  202  10/10/1996 (27)   5
5             Austin Deculus   OT   6-5  321   3/12/1999 (25)   3
6             Austin Seibert    K   5-9  200  11/15/1996 (27)   4
7               Brady Latham   OG   6-5  312   9/30/2000 (23)   R
8              Braelon Allen   RB   6-1  235   1/20/2004 (20)   R
9           Braiden McGregor   DE   6-6  267   7/12/2001 (22)   R
10            Brandin Echols   CB  5-10  179  10/16/1997 (26)   4
11        Brandon Codrington   DB   5-9  185                —   R
12               Breece Hall   RB  5-11  217   5/31/2001 (23)   3
13              Bruce Hector   DT   6

Request succeeded with status code 200
                        Player  POS    HT   WT        DOB (AGE) EXP
0                 Alex Barrett   DE   6-2  250    3/6/1994 (30)   2
1                 Ambry Thomas   CB   6-0  190    9/9/1999 (24)   4
2                Austin Bryant   DE   6-5  250  11/12/1996 (27)   5
3                   Ben Bartch   OG   6-6  315   7/22/1998 (25)   5
4                Brandon Aiyuk   WR   6-0  200   3/17/1998 (26)   5
5                Brandon Allen   QB   6-2  209    9/5/1992 (31)   8
6               Brandon Parker   OT   6-8  320  10/21/1995 (28)   7
7               Brayden Willis   TE   6-4  240  11/11/1999 (24)   2
8                 Briason Mays    C   6-4  295                —   R
9                  Brock Purdy   QB   6-1  220  12/27/1999 (24)   3
10                Cameron Latu   TE   6-5  244   2/24/2000 (24)   2
11             Charvarius Ward   CB   6-1  196   5/16/1996 (28)   7
12                 Chase Lucas   CB  5-11  184    3/4/1997 (27)   3
13       

                 Player  POS    HT   WT        DOB (AGE) EXP
0          Andrew Hayes   CB   6-0  195   5/17/2001 (23)   R
1        Anthony Nelson   LB   6-7  271    3/4/1997 (27)   6
2   Antoine Winfield Jr  SAF   5-9  203   8/16/1998 (25)   5
3         Antonio Grier   LB   6-1  230   1/18/2000 (24)   R
4           Avery Jones    C   6-4  307  10/20/1999 (24)   R
5        Baker Mayfield   QB   6-1  215   4/14/1995 (29)   7
6          Ben Bredeson   OG   6-5  315   2/20/1998 (26)   5
7        Brandon Walton   OT   6-5  300   3/18/1998 (26)   3
8            Bryce Hall   CB   6-1  202   11/5/1997 (26)   5
9          Bucky Irving   RB  5-10  194   8/19/2002 (21)   R
10          C.J. Brewer   DT   6-2  290  11/22/1997 (26)   1
11           Cade Otton   TE   6-5  247   4/15/1999 (25)   3
12       Calijah Kancey   DT   6-0  280    3/1/2001 (23)   2
13   Cephus Johnson III   WR   6-5  220   9/21/1999 (24)   1
14        Chase Edmonds   RB   5-9  201   4/13/1996 (28)   7
15     Chase McLaughlin 

Request succeeded with status code 200
                     Player  POS    HT   WT        DOB (AGE) EXP
0                A.J. Woods   DB   5-9  186   3/28/2001 (23)   R
1            Alex Akingbulu   OT   6-5  300  12/28/1997 (26)   1
2               Andre Jones   DE   6-5  258  10/25/1998 (25)   2
3              Andrew Wylie   OT   6-6  309   8/19/1994 (29)   7
4           Anthony Pittman   LB   6-3  224  11/24/1996 (27)   4
5             Armani Rogers   TE   6-5  225   12/4/1997 (26)   3
6             Austin Ekeler   RB  5-10  200   5/17/1995 (29)   8
7              Austin Jones   RB  5-10  200    2/7/2001 (23)   R
8                Ben Nikkel   DB   6-1  210                —   R
9               Ben Sinnott   TE   6-4  247   6/14/2002 (22)   R
10        Benjamin St-Juste   CB   6-3  200    9/8/1997 (26)   4
11          Benning Potoa'e   DE   6-3  290   9/17/1996 (27)   1
12                 Bo Bauer   LB   6-3  253                —   R
13             Bobby Wagner  MLB   6-0  242   6/27/